<a href="https://colab.research.google.com/github/silverstar0727/1day-1commit-challenge/blob/master/bitcoin_data_down.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# google drive와 연동
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# 연동 테스트 코드
with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')

!cat /content/gdrive/My\ Drive/foo.txt

Hello Google Drive!

In [14]:
# 출처: https://gry1994.tistory.com/225

import requests
from datetime import datetime
import pandas as pd
import time
 
starting = time.time()
 
ep='https://api.binance.com'
candle = '/api/v3/klines'
 
start_date = []
start_time = []
openn = []
high = []
low = []
close = []
volume = []
 
start = 1504224000000 # 17년 9월 1일 09:00:00 최초날짜
#start = 1582748490000 # 테스트를 위해 최근만 받아옴 (20년 2월 27일~)
first_params_candle = {'symbol': 'BTCUSD', 'interval' : '1m','startTime' : start , 'limit' : 1 } # 초기세팅
r1 = requests.get(ep+candle, params=first_params_candle )
 
while len(r1.json()) > 0 :
    first_params_candle = {'symbol': 'BTCUSD', 'interval' : '1m','startTime' : start , 'limit' : 1000 }
    r1 = requests.get(ep+candle, params=first_params_candle ) #use parameter
    req = r1.json()
    for i in range(0,len(req)) :
        start_date.append(datetime.fromtimestamp(req[i][0]/1000).strftime('%Y%m%d'))
        start_time.append(datetime.fromtimestamp(req[i][0]/1000).strftime('%H%M'))
        openn.append(req[i][1])
        high.append(req[i][2])
        low.append(req[i][3])
        close.append(req[i][4])
        volume.append(req[i][5])
    if len(req) > 0 : # 받아온 데이터가 있을 경우 => 다음 루프 시작
        start = req[-1][6]+1
        print(datetime.fromtimestamp(req[-1][6]/1000).strftime('%Y%m%d'), "데이터 다운로드 완료...")
    else : print("다운로드 완료")
    #time.sleep(0.25) # 횟수제한을 위해 추가
 
ticker = ['BTCUSD'] * len(start_date)
 
chartData = {'date': start_date, 'time': start_time, 'ticker': ticker, 'open': openn, 'high': high,
             'low': low, 'close': close, 'volume': volume}
df = pd.DataFrame(chartData,
                  columns=['date', 'time', 'ticker', 'open', 'high', 'low', 'close', 'volume'])
 
 
df.to_csv("/content/gdrive/My Drive/BTCUSD_1minute.csv", header = True, index = False) # csv파일로 저장
 
print("소요 시간 : ", time.time() - starting)

KeyError: ignored

In [0]:
train = pd.read_csv('/content/gdrive/My Drive/BTCUSDT_3minute.csv')

In [13]:
train

,date,time,ticker,open,high,low,close,volume
0,20200226,2024,BTCUSDT,8858.64,8860.52,8843.64,8846.19,96.790884
1,20200226,2027,BTCUSDT,8846.19,8864.00,8843.65,8861.67,95.119714
2,20200226,2030,BTCUSDT,8861.68,8861.68,8830.00,8834.79,107.558519
3,20200226,2033,BTCUSDT,8833.26,8844.91,8824.02,8838.83,104.986623
4,20200226,2036,BTCUSDT,8838.71,8841.80,8809.09,8812.09,132.453956
...,...,...,...,...,...,...,...,...
17940,20200404,730,BTCUSDT,6716.51,6724.70,6715.94,6721.08,30.712223
17941,20200404,733,BTCUSDT,6721.08,6727.50,6719.73,6724.06,44.774862
17942,20200404,736,BTCUSDT,6724.06,6727.70,6717.72,6723.80,73.958007
17943,20200404,739,BTCUSDT,6723.80,6725.57,6718.11,6723.03,33.266379


In [18]:
!pip install websocket-client

     |████████████████████████████████| 204kB 2.8MB/s 


In [0]:
!pip install websocket

In [2]:

import websocket
import requests
import json

# 중요한 값은 상수사용합니다.
SYMBOL_LIST_ENDPOINT = "https://www.bitmex.com/api/v1/instrument/active"
ENDPOINT = 'wss://www.bitmex.com/realtime'

def sendRequest(url):
    res = requests.get(url)
    contents = res.content.decode('utf-8')
    # python2에서는 contents = res.content만 적어도 됩니다.
    # python2에서는 res.content가 string으로 오지만
    # python3에서는 byte로 오기 때문에 디코딩이 필요합니다.

    # 대부분의 api는 json으로 반환되기 때문에 곧바로 json 라이브러리를 이용해 로드합니다
    json_contents = json.loads(contents)
    return json_contents

def getBitmexSymbolList(contents):
    symbol_list = [
          pair['symbol']
          for pair in contents
         ]
    # contents안에 들어있는 각각의 아이템들 중에서
    # 키가 symbol인 것들만 리스트에 넣겠다는 뜻

    formatted_list = [ 'tradeBin1m:{}'.format(symbol) for symbol in symbol_list]
    #  {"op":"subscribe", "args" : []}의 형식이었죠? args에 formatted_list를
    #  웹서버로 보내주면 bitmex에서 제공하는 모든 symbol에 대한 OHLCV를 얻을 수 있습니다.

    return {"op":"subscribe", "args" : formatted_list}

def on_message(ws, message):
    print(message)

def on_error(ws, error):
    print(error)
    ws.on_close(ws)

def on_close(ws):
    print("### closed ###")
    ws.close()

def on_open(ws):
    print("### open ###")
    contents = sendRequest(SYMBOL_LIST_ENDPOINT)
    symbols = getBitmexSymbolList(contents)
    ws.send(json.dumps(symbols))



def run(endpoint):
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp(endpoint,
                                on_open = on_open,
                                on_message = on_message,
                                on_error = on_error,
                                on_close = on_close)

    ws.run_forever()

if __name__ == "__main__":
    run(ENDPOINT)

--- request header ---
GET /realtime HTTP/1.1
Upgrade: websocket
Host: www.bitmex.com
Origin: http://www.bitmex.com
Sec-WebSocket-Key: g0kaNCmN+2/6nxXAkYG7YQ==
Sec-WebSocket-Version: 13
Connection: upgrade


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Sat, 04 Apr 2020 08:18:45 GMT
Connection: upgrade
Set-Cookie: AWSALBTG=/gpaMdoM4MgLun0vjgkByvS0ev+tCxgylErI3ESJ/8oqQqmSvEj+zAJ7z0/u6Gsts46ywtjjajatZdRiC1yS1o5sBFao/PYUhdI6xaueun/grMOlkOLkmHDFJovsDzYlIjpvo/yEIzCQAaCj/CL2AXWTVJH1W+m6BwMWWgJB+0YBMApLto8=; Expires=Sat, 11 Apr 2020 08:18:45 GMT; Path=/
Set-Cookie: AWSALBTGCORS=/gpaMdoM4MgLun0vjgkByvS0ev+tCxgylErI3ESJ/8oqQqmSvEj+zAJ7z0/u6Gsts46ywtjjajatZdRiC1yS1o5sBFao/PYUhdI6xaueun/grMOlkOLkmHDFJovsDzYlIjpvo/yEIzCQAaCj/CL2AXWTVJH1W+m6BwMWWgJB+0YBMApLto8=; Expires=Sat, 11 Apr 2020 08:18:45 GMT; Path=/; SameSite=None; Secure
Set-Cookie: AWSALB=EbhVkhNyGlUo5eAiU4p7G3zN0B8dEoUFqXXZAWtUmZ4gGSJ8zxVx6pwbqYclLhGIQ4+W+/Baz35jd8nG7Hz6UDDlmsG1oXiF+ItXJibniXOGlov

### open ###


send: b'\x81\xfe\x01\x19\xa0\xec\xf2\xbf\xdb\xce\x9d\xcf\x82\xd6\xd2\x9d\xd3\x99\x90\xcc\xc3\x9e\x9b\xdd\xc5\xce\xde\x9f\x82\x8d\x80\xd8\xd3\xce\xc8\x9f\xfb\xce\x86\xcd\xc1\x88\x97\xfd\xc9\x82\xc3\xd2\x9a\xb4\xa0\xef\xed\xde\xc2\x9d\x8c\xcc\xd0\xcb\xd2\x8d\x96\xda\xe2\x85\x9c\x8e\xcd\xd6\xb0\xfc\xe8\xa1\xc0\x8f\x82\xc0\xd2\x9d\xd4\x9e\x93\xdb\xc5\xae\x9b\xd1\x91\x81\xc8\xfe\xe4\xad\xbf\x8d\x90\xce\xde\x9f\x82\x98\x80\xde\xc4\x89\xb0\xd6\xce\xdd\x9f\x85\xe5\xa3\xa1\xf2\x92\xdc\xd0\x93\x80\xce\x86\xcd\xc1\x88\x97\xfd\xc9\x82\xc3\xd2\x9a\xb8\xa0\xe7\xed\xde\xc2\x9d\x8c\xcc\xd0\xcb\xd2\x8d\x96\xda\xe2\x85\x9c\x8e\xcd\xd6\xaa\xed\xf0\xb9\xa1\xfb\x82\xc0\xd2\x9d\xd4\x9e\x93\xdb\xc5\xae\x9b\xd1\x91\x81\xc8\xe7\xe2\xb8\xa7\xec\xe4\xce\xde\x9f\x82\x98\x80\xde\xc4\x89\xb0\xd6\xce\xdd\x9f\x85\xf8\xae\xa6\xf2\x92\xdc\xd0\x93\x80\xce\x86\xcd\xc1\x88\x97\xfd\xc9\x82\xc3\xd2\x9a\xb4\xb0\xeb\xf5\xde\xc2\x9d\x8c\xcc\xd0\xcb\xd2\x8d\x96\xda\xe2\x85\x9c\x8e\xcd\xd6\xb7\xeb\xe8\xb9\xa1\xfb\x82\xc0\xd2\x9d

{"info":"Welcome to the BitMEX Realtime API.","version":"2020-03-25T22:49:58.000Z","timestamp":"2020-04-04T08:18:45.172Z","docs":"https://www.bitmex.com/app/wsAPI","limit":{"remaining":39}}
{"table":"tradeBin1m","action":"partial","keys":[],"types":{"timestamp":"timestamp","symbol":"symbol","open":"float","high":"float","low":"float","close":"float","trades":"long","volume":"long","vwap":"float","lastSize":"long","turnover":"long","homeNotional":"float","foreignNotional":"float"},"foreignKeys":{"symbol":"instrument"},"attributes":{"timestamp":"sorted","symbol":"grouped"},"filter":{"symbol":"XRPM20"},"data":[{"timestamp":"2020-04-04T08:18:00.000Z","symbol":"XRPM20","open":0.00002664,"high":0.00002664,"low":0.00002664,"close":0.00002664,"trades":0,"volume":0,"vwap":null,"lastSize":null,"turnover":0,"homeNotional":0,"foreignNotional":0}]}
{"table":"tradeBin1m","action":"partial","keys":[],"types":{"timestamp":"timestamp","symbol":"symbol","open":"float","high":"float","low":"float","close

send: b'\x88\x82\xf6\xe4\xa4\x89\xf5\x0c'



### closed ###
### closed ###
